In [1]:
import sys
sys.path.append(r"D:\Study\Intelligence And Learning\Fluid Mechanics\Project 2\Code\NNAero")
from nnaero.models.vae import *
from nnaero.models.training.vae_trainer import *

## Loading Datasets

In [7]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# 1. Initialize your model
import torch.optim.adamw


FEAT_SIZE = 257  # As per your code
model = VAE(feature_size=FEAT_SIZE, latent_size=32)

BATCH_SIZE = 64

# Make a good dataset
mock_data = torch.randn(1024, FEAT_SIZE)
dataset = TensorDataset(mock_data)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)











# 2. Define Configuration
# config = VAETrainingConfig(
#     lr=5e-4, 
    # batch_size=64, 
    # epochs=100,
    # kl_annealing=True,
    # kl_warmup_epochs=20
# )

# 3. Create Trainer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
trainer = VAETrainer(model,
                     optimizer=optimizer,
                    kl_annealing=True,
                    warmup_epochs=20,
                    checkpoint_dir="vae_checkpoints"
                     )

In [16]:
EPOCHS=100

history = []
for i in range(EPOCHS):
    history.append(trainer.train_epoch(train_loader, i))
    print(f"For epochs {i}: loss: {history[i]['loss']}")

For epochs 0: loss: 252.9343032836914
For epochs 1: loss: 251.89337635040283
For epochs 2: loss: 251.0240821838379
For epochs 3: loss: 250.31494235992432
For epochs 4: loss: 250.03369045257568
For epochs 5: loss: 249.8411464691162
For epochs 6: loss: 249.6790361404419
For epochs 7: loss: 249.85372734069824
For epochs 8: loss: 249.85483074188232
For epochs 9: loss: 249.97079944610596
For epochs 10: loss: 250.07574653625488
For epochs 11: loss: 250.24957370758057
For epochs 12: loss: 250.5425443649292
For epochs 13: loss: 250.74553298950195
For epochs 14: loss: 250.6940402984619
For epochs 15: loss: 251.1576919555664
For epochs 16: loss: 251.28039455413818
For epochs 17: loss: 251.70767211914062
For epochs 18: loss: 251.77567100524902
For epochs 19: loss: 252.0239381790161
For epochs 20: loss: 252.36128902435303
For epochs 21: loss: 252.05591297149658
For epochs 22: loss: 251.9017219543457
For epochs 23: loss: 251.8402853012085
For epochs 24: loss: 251.75771522521973
For epochs 25: loss:

In [15]:
history[0]["loss"]

258.3489294052124